In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
%matplotlib inline
from scipy.interpolate import interp1d
from numpy import trapz

In [ ]:
# Chemin du répertoire + nom du fichier
chemin = r'C:\Users\navid.mouhamad\OneDrive - IPVF\Desktop\Stages 2025\1ere_campagne\XRD_campagne_1.csv'
df = pd.read_csv(chemin, sep=';')
df

In [ ]:
x = df.iloc[:, 0]
y = df.iloc[:, 1]

interpolation = interp1d(x, y, kind='linear', fill_value='extrapolate')

# Borne d'intégration 1 = PbI2
x_inter = np.linspace(11.7, 12.8, 1000)
y_inter = interpolation(x_inter)
aire = trapz(y_inter, x_inter)
print("Intégrale de la courbe interpolée :", aire)

# Borne d'intégration 2 = PSK
x_inter2 = np.linspace(13, 14.2, 1000)
y_inter2 = interpolation(x_inter2)
aire2 = trapz(y_inter2, x_inter2)
print("Intégrale de la courbe interpolée :", aire2)

plt.plot(x, y, '-', label='Données expérimentales')

plt.plot(x_inter, y_inter,'r-', label='Intégrale PbI2')
plt.fill_between(x_inter, y_inter, alpha=0.3)

plt.plot(x_inter2, y_inter2,'r-', label='Intégrale PSK')
plt.fill_between(x_inter2, y_inter2, alpha=0.3)

plt.legend()
plt.grid()
plt.show()

In [ ]:
integrale = []
gap = []
fwhm_list = []
max_pic = []

integrale2 = []
gap2 = []
fwhm_list2 = []
max_pic2 = []

label_graph = []

for i in range(1, len(df.columns), 2):
    y  = df.iloc[:, i] # intensity
    x = df.iloc[:, 0]  # degree
    interpolation = interp1d(x, y, kind='linear', fill_value='extrapolate')
    #x_inter = np.linspace(11.5, 13, 1000)
    #x_inter2 = np.linspace(13, 14.5, 1000)
    y_inter = interpolation(x_inter)
    y_inter2 = interpolation(x_inter2)
    
    # Integrale
    aire = trapz(y_inter, x_inter)
    aire2 = trapz(y_inter2, x_inter2)
    print("Intégrale de la courbe interpolée PbI2 :", aire)
    integrale.append(aire)
    print("Intégrale de la courbe interpolée PSK :", aire2)
    integrale2.append(aire2)
    
    # 2θ du pic de PbI2
    val_gap = np.argmax(y_inter)
    x_max = x_inter[val_gap]
    print("2θ PbI2 :", x_max)
    gap.append(x_max)
    
    # 2θ du pic de PSK
    val_gap2 = np.argmax(y_inter2)
    x_max2 = x_inter2[val_gap2]
    print("2θ PSK :", x_max2)
    gap2.append(x_max2)
    
    # Counts du pic de PbI2
    val_counts = np.argmax(y_inter)
    y_max = y_inter[val_counts]
    print("Counts max PbI2 :", y_max)
    max_pic.append(y_max)
    
    # Counts du pic de PSK
    val_counts2 = np.argmax(y_inter2)
    y_max2 = y_inter2[val_counts2]
    print("Counts max PSK :", y_max2)
    max_pic2.append(y_max2)
    
    # FWHM PbI2
    y_max = np.max(y_inter)
    half_max = y_max / 2
    indices_above_half_max = np.where(y_inter >= half_max)[0]
    x_left = x_inter[indices_above_half_max[0]]
    x_right = x_inter[indices_above_half_max[-1]]
    fwhm = x_right - x_left
    print("FWHM PbI2 :", fwhm)
    fwhm_list.append(fwhm)
    
    # FWHM PSK
    y_max2 = np.max(y_inter2)
    half_max2 = y_max2 / 2
    indices_above_half_max2 = np.where(y_inter2 >= half_max2)[0]
    x_left2 = x_inter2[indices_above_half_max2[0]]
    x_right2 = x_inter2[indices_above_half_max2[-1]]
    fwhm2 = x_right2 - x_left2
    print("FWHM PSK :", fwhm2)
    fwhm_list2.append(fwhm2)
    
    val_label = df.columns[i]
    label_graph.append(val_label)
    
    plt.plot(x, y,'-', label=df.columns[i])
    
    plt.plot(x_inter, y_inter,'r-', label='Intégrale PbI2')
    plt.fill_between(x_inter, y_inter, alpha=0.3)
    
    plt.plot(x_inter2, y_inter2,'r-', label='Intégrale PSK')
    plt.fill_between(x_inter2, y_inter2, alpha=0.3)
    
    plt.legend()
    plt.show()
    

In [ ]:
# % de PbI2 dans la PSK (integrale)
percent_integrale = []
percent_integrale = (np.array(integrale)*100)/(np.array(integrale)+np.array(integrale2))
#percent_integrale 

# % de PbI2 dans la PSK (intensité du pic)
percent_int_pic = []
percent_int_pic = (np.array(max_pic)*100)/(np.array(max_pic)+np.array(max_pic2))
#percent_int_pic  

# % de PbI2 dans la PSK (FWHM)
percent_fwhm = []
percent_fwhm = (np.array(fwhm_list)*100)/(np.array(fwhm_list)+np.array(fwhm_list2))
#percent_fwhm  

In [ ]:
# Sauvegarde dans un dataframe "XRD_treated"
XRD_treated = pd.DataFrame()

XRD_treated['Label'] = label_graph
XRD_treated['Integrale PbI2'] = integrale
XRD_treated['Integrale PSK'] = integrale2
XRD_treated['%PbI2 integrale'] = percent_integrale
XRD_treated['Pic PbI2 2θ (°)'] = gap
XRD_treated['Pic PSK 2θ (°)'] = gap2
XRD_treated['FWHM PbI2 (°)'] = fwhm_list
XRD_treated['FWHM PSK (°)'] = fwhm_list2
XRD_treated['%PbI2 FWHM'] = percent_fwhm 
XRD_treated['Counts max PbI2 (a.u.)'] = max_pic
XRD_treated['Counts max PSK (a.u.)'] = max_pic2
XRD_treated['%PbI2 counts'] = percent_int_pic 

In [ ]:
# Conversion en csv (dans le dossier ou se situe le code)
XRD_treated.to_csv('XRD_treated.csv', index=False)
XRD_treated